# Importing libraries

In [7]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
import folium
import json

# Importing libraries

In [8]:
country_geo=r'/Users/urvi/Desktop/ACHIVEMENT 6/02 Data /us-states.json'

In [9]:
country_geo

'/Users/urvi/Desktop/ACHIVEMENT 6/02 Data /us-states.json'

In [10]:
path=r'/Users/urvi/Desktop/ACHIVEMENT 6/02 Data /Prepared Data '
path

'/Users/urvi/Desktop/ACHIVEMENT 6/02 Data /Prepared Data '

In [11]:
df=pd.read_csv(os.path.join(path, 'insurance_cleaned.csv'))

# Data Wrangling

### Inspeacting Dataset

In [12]:
df.head()

,Unnamed: 0,age,sex,bmi,children,smoker,region,charges
0,0,19,female,27.900,0,yes,southwest,16884.92400
1,1,18,male,33.770,1,no,southeast,1725.55230
2,2,28,male,33.000,3,no,southeast,4449.46200
3,3,33,male,22.705,0,no,northwest,21984.47061
4,4,32,male,28.880,0,no,northwest,3866.85520


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1337 entries, 0 to 1336
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  1337 non-null   int64  
 1   age         1337 non-null   int64  
 2   sex         1337 non-null   object 
 3   bmi         1337 non-null   float64
 4   children    1337 non-null   int64  
 5   smoker      1337 non-null   object 
 6   region      1337 non-null   object 
 7   charges     1337 non-null   float64
dtypes: float64(2), int64(3), object(3)
memory usage: 83.7+ KB


### Checking for missing values

In [14]:
df.isnull().sum()

Unnamed: 0    0
age           0
sex           0
bmi           0
children      0
smoker        0
region        0
charges       0
dtype: int64

### Data Type Conversion

In [15]:
df['smoker'] = df['smoker'].map({'yes': 1, 'no': 0})

### One-hot coding

In [16]:
df = pd.get_dummies(df, columns=['region'], drop_first=True)

In [17]:
df.head()

,Unnamed: 0,age,sex,bmi,children,smoker,charges,region_northwest,region_southeast,region_southwest
0,0,19,female,27.900,0,1,16884.92400,False,False,True
1,1,18,male,33.770,1,0,1725.55230,False,True,False
2,2,28,male,33.000,3,0,4449.46200,False,True,False
3,3,33,male,22.705,0,0,21984.47061,True,False,False
4,4,32,male,28.880,0,0,3866.85520,True,False,False


### Normalization

In [18]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
df[['age', 'bmi', 'charges']] = scaler.fit_transform(df[['age', 'bmi', 'charges']])

### Creating or Dropping Columns

In [19]:
df['overweight'] = (df['bmi'] >= 25).astype(int)

In [20]:
df.drop(columns=['Unnamed: 0'], inplace=True)

In [21]:
df.head()

,age,sex,bmi,children,smoker,charges,region_northwest,region_southeast,region_southwest,overweight
0,-1.440418,female,-0.453160,0,1,0.297857,False,False,True,0
1,-1.511647,male,0.509422,1,0,-0.954381,False,True,False,0
2,-0.799350,male,0.383155,3,0,-0.729373,False,True,False,0
3,-0.443201,male,-1.305052,0,0,0.719104,True,False,False,0
4,-0.514431,male,-0.292456,0,0,-0.777499,True,False,False,0


### Handling outliers

In [22]:
Q1 = df['charges'].quantile(0.25)
Q3 = df['charges'].quantile(0.75)
IQR = Q3 - Q1

In [23]:
# Filtering out outliers
df_filtered = df[~((df['charges'] < (Q1 - 1.5 * IQR)) |(df['charges'] > (Q3 + 1.5 * IQR)))]

In [24]:
df_filtered

,age,sex,bmi,children,smoker,charges,region_northwest,region_southeast,region_southwest,overweight
0,-1.440418,female,-0.453160,0,1,0.297857,False,False,True,0
1,-1.511647,male,0.509422,1,0,-0.954381,False,True,False,0
2,-0.799350,male,0.383155,3,0,-0.729373,False,True,False,0
3,-0.443201,male,-1.305052,0,0,0.719104,True,False,False,0
4,-0.514431,male,-0.292456,0,0,-0.777499,True,False,False,0
...,...,...,...,...,...,...,...,...,...,...
1332,0.767704,male,0.050269,3,0,-0.221263,True,False,False,0
1333,-1.511647,female,0.206053,0,0,-0.914695,False,False,False,0
1334,-1.511647,female,1.014490,0,0,-0.962287,False,True,False,0
1335,-1.297958,female,-0.797524,0,0,-0.931054,False,False,True,0


I've meticulously wrangled the insurance dataset to prepare it for a comprehensive analysis. My initial step involved a thorough inspection of the dataset, confirming its well-structured nature with no missing values, setting a solid foundation for my analysis. I then transformed the 'smoker' column from categorical to numerical, aiming to quantify its impact on insurance charges more precisely. By applying one-hot encoding to the 'region' variable, I've expanded my analytical capabilities, allowing me to dissect the influence of regional factors with increased accuracy.

To ensure comparability across different scales, I normalized 'age', 'bmi', and 'charges' using the StandardScaler, effectively leveling the playing field for these variables. Recognizing the potential significance of BMI in the context of insurance charges, I introduced an 'overweight' binary indicator, adding another layer of depth to my analysis. Additionally, I removed the redundant 'Unnamed: 0' column, streamlining the dataset for ease of use.

Anticipating the potential distortion caused by outliers, especially within the 'charges' variable, I calculated the Interquartile Range (IQR) to identify and address these extremes. This meticulous approach to data wrangling not only refines the dataset but also aligns it perfectly with my analytical objectives. I'm now poised to delve into exploratory data analysis, hypothesis testing, and predictive modeling, eager to unearth insights that could shed light on the factors influencing insurance charges and contribute to the broader discourse on insurance policies and health economics.

# Cleaning Data set

In [25]:
# Check again for any missing values across all columns
missing_values = df.isnull().sum()
print(missing_values)

age                 0
sex                 0
bmi                 0
children            0
smoker              0
charges             0
region_northwest    0
region_southeast    0
region_southwest    0
overweight          0
dtype: int64


### Calculate IQR for 'charges'

In [26]:
Q1 = df['charges'].quantile(0.25)
Q3 = df['charges'].quantile(0.75)
IQR = Q3 - Q1

# Define boundaries for outliers
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Filter out outliers
df_filtered_charges = df[(df['charges'] >= lower_bound) & (df['charges'] <= upper_bound)]

In [27]:
original_count = df.shape[0]

# Count after filtering out BMI outliers
filtered_count = df_filtered_charges.shape[0]

# Calculate the number of rows (data points) excluded as outliers
excluded_count = original_count - filtered_count

# Calculate the percentage of data excluded
excluded_percentage = (excluded_count / original_count) * 100

print(f"Original number of data points: {original_count}")
print(f"Number of data points after filtering: {filtered_count}")
print(f"Number of data points excluded as outliers: {excluded_count}")
print(f"Percentage of data excluded: {excluded_percentage:.2f}%")

Original number of data points: 1337
Number of data points after filtering: 1198
Number of data points excluded as outliers: 139
Percentage of data excluded: 10.40%


### Calculate IQR for 'bmi'

In [28]:
# Calculate the IQR for BMI
Q1_bmi = df['bmi'].quantile(0.25)
Q3_bmi = df['bmi'].quantile(0.75)
IQR_bmi = Q3_bmi - Q1_bmi

# Define boundaries for outliers in BMI
lower_bound_bmi = Q1_bmi - 1.5 * IQR_bmi
upper_bound_bmi = Q3_bmi + 1.5 * IQR_bmi

# Filtering out outliers from the DataFrame based on BMI
df_filtered_bmi = df[(df['bmi'] >= lower_bound_bmi) & (df['bmi'] <= upper_bound_bmi)]

In [29]:
original_count = df.shape[0]

# Count after filtering out BMI outliers
filtered_count = df_filtered_bmi.shape[0]

# Calculate the number of rows (data points) excluded as outliers
excluded_count = original_count - filtered_count

# Calculate the percentage of data excluded
excluded_percentage = (excluded_count / original_count) * 100

print(f"Original number of data points: {original_count}")
print(f"Number of data points after filtering: {filtered_count}")
print(f"Number of data points excluded as outliers: {excluded_count}")
print(f"Percentage of data excluded: {excluded_percentage:.2f}%")

Original number of data points: 1337
Number of data points after filtering: 1328
Number of data points excluded as outliers: 9
Percentage of data excluded: 0.67%


After thoroughly cleaning the dataset, I focused on outliers in 'charges' and 'bmi', crucial for my analysis. Removing 139 outliers from 'charges'—10.40% of the dataset—was significant, though it meant possibly overlooking rare, high-cost scenarios. In contrast, only 9 'bmi' outliers were excluded, affecting just 0.67% of the dataset, indicating a consistent and expected distribution among the population.

The difference in outlier percentages between 'charges' and 'bmi' highlights their distinct natures—'charges' show wide variability, reflecting the diverse landscape of healthcare costs, while 'bmi' is more uniformly distributed. This process has sharpened the dataset, preparing it for an in-depth analysis that will explore healthcare costs without being skewed by extreme values. Going forward, I plan to balance the general analysis with specific attention to 'charges' outliers to ensure comprehensive insights.

In [30]:
df.head()

,age,sex,bmi,children,smoker,charges,region_northwest,region_southeast,region_southwest,overweight
0,-1.440418,female,-0.453160,0,1,0.297857,False,False,True,0
1,-1.511647,male,0.509422,1,0,-0.954381,False,True,False,0
2,-0.799350,male,0.383155,3,0,-0.729373,False,True,False,0
3,-0.443201,male,-1.305052,0,0,0.719104,True,False,False,0
4,-0.514431,male,-0.292456,0,0,-0.777499,True,False,False,0


# Exoporting dataset

In [31]:
df.to_csv(os.path.join(path,'insurance_final.csv'))

# Create a choropleth map using a variable of your choice

In [32]:
# Initialize your map
m = folium.Map(location=[40, -95], zoom_start=4)

In [33]:
df.columns

Index(['age', 'sex', 'bmi', 'children', 'smoker', 'charges',
       'region_northwest', 'region_southeast', 'region_southwest',
       'overweight'],
      dtype='object')

In [34]:
def assign_region(row):
    if row['region_northwest']:
        return 'Northwest'
    elif row['region_southeast']:
        return 'Southeast'
    elif row['region_southwest']:
        return 'Southwest'
    else:
        return 'Northeast'

# Apply the function to create a new 'region' column
df['region'] = df.apply(assign_region, axis=1)

# Now, calculate the average charges by region
average_charges_by_region = df.groupby('region')['charges'].mean()

In [35]:
average_charges_by_region

region
Northeast    0.010513
Northwest   -0.068420
Southeast    0.120297
Southwest   -0.077003
Name: charges, dtype: float64

In [36]:
folium.Choropleth(
    geo_data=country_geo,
    name='choropleth',
    data=df,
    columns=['region', 'charges'],
    key_on='feature.properties.name',  # Adjust based on your GeoJSON's structure
    fill_color='YlGnBu',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Average Insurance Charges by Region'
).add_to(m)

folium.LayerControl().add_to(m)
m

In [55]:
# Exporting the map
m.save('/Users/urvi/Desktop/ACHIVEMENT 6/04 Analysis /Visuals/inschrgregion_output.html')

In [37]:
region_to_states = {
    'Northeast': ['ME', 'NH', 'VT', 'MA', 'RI', 'CT', 'NY', 'NJ', 'PA'],
    'Southeast': ['DE', 'MD', 'VA', 'WV', 'KY', 'TN', 'NC', 'SC', 'GA', 'AL', 'MS', 'AR', 'LA', 'FL'],
    'Southwest': ['TX', 'OK', 'NM', 'AZ'],
    'Northwest': ['WA', 'OR', 'ID', 'MT', 'WY']}

charges_by_region = {
    'Northeast':0.010513,
    'Northwest':-0.068420,
    'Southeast':0.120297,
    'Southwest':-0.077003,
}

In [39]:
# Create a list of dictionaries, each representing a state and its assigned charge
state_charges = []
for region, states in region_to_states.items():
    for state in states:
        state_charges.append({'State': state, 'Charges': charges_by_region[region]})

# Convert list to DataFrame
state_charges_df = pd.DataFrame(state_charges)

In [40]:
m = folium.Map(location=[37.8, -96], zoom_start=4)

# Add the choropleth layer
folium.Choropleth(
    geo_data=country_geo,
    name='choropleth',
    data=state_charges_df,
    columns=['State', 'Charges'],
    key_on='feature.id',  # Ensure this matches your GeoJSON structure
    fill_color='YlOrRd',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Average Insurance Charges by State'
).add_to(m)

folium.LayerControl().add_to(m)

# Display the map
m

The second map presents the average insurance charges by state. States with darker shades are likely to have higher average insurance charges. It seems that the Southeast, particularly states like Florida and Georgia, have higher charges, as indicated by the redder hues. In contrast, states in the Northwest like Oregon and Washington exhibit much lighter shades, suggesting lower average insurance charges.

In [48]:
# Exporting the map
m.save('/Users/urvi/Desktop/ACHIVEMENT 6/04 Analysis /Visuals/avgchrgins_output.html')

In [49]:
# Calculate the proportion of smokers in each region
smoker_proportions = df.groupby('region')['smoker'].mean().reset_index()

In [50]:
df['smoker'] = pd.to_numeric(df['smoker'], errors='coerce')

In [51]:
state_smoker_proportions = {state: 0 for state in country_geo}  # Initialize all states with 0

In [52]:
for _, row in smoker_proportions.iterrows():
    region = row['region']
    proportion = row['smoker']
    for state in region_to_states[region]:
        state_smoker_proportions[state] = proportion

In [53]:
folium.Choropleth(
    geo_data=country_geo,
    name='choropleth',
    data=pd.DataFrame.from_dict(state_smoker_proportions, orient='index').reset_index(),
    columns=['index', 0],
    key_on='feature.id',  # Ensure this matches the key in your GeoJSON
    fill_color='YlGnBu',  # Choose a color scheme
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Proportion of Smokers by Region'
).add_to(m)

folium.LayerControl().add_to(m)
m

The above map appears to illustrate the proportion of smokers by region within the United States. Regions with darker shades suggest a higher prevalence of smokers. If we look closely, the Southeast has the darkest shade, which might indicate that this region has the highest proportion of smokers compared to the Northeast, Northwest, and Southwest. This could lead to questions about regional lifestyle differences, public health policies, or socioeconomic factors contributing to this distribution.

These maps could lead to several new research questions:

What are the underlying causes of regional differences in smoking prevalence?

How do state-specific healthcare policies and programs affect insurance charges?

Is there a causal relationship between smoking prevalence and healthcare costs, or do other factors (e.g., obesity, pollution) play a more significant role?

What strategies have been successful in regions with lower smoking rates, and can they be adapted to regions with higher rates?

In [54]:
# Exporting the map
m.save('/Users/urvi/Desktop/ACHIVEMENT 6/04 Analysis /Visuals/avginschrstate_output.html')